Генерация данных:

Создать DataFrame с полями: Дата, UserID, Продукт, Количество, Цена.

Данные для поля Продукт генерируются из списка возможных товаров ( не меньше 5 товаров )

Количество и Цена должны генерироваться случайно в заданных пределах.

Дата должна быть в пределах последнего года.

UserID представляет собой случайное число, имитирующее идентификаторы пользователей.

Обратите внимание, что должна быть возможности изменять количество сгенерированных строк. Минимальное количество - 1000 строк.

Сохранение данных:

Сохранить сгенерированный DataFrame в формате CSV для последующего анализа.


In [2]:
pip install pyspark


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 3.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.3-py2.py3-none-any.whl size=317840625 sha256=6e5a15d1761b5db345cccb9168457467ef592f995f8db9d53502fe6f2b2b612a
  Stored in directory: /root/.cache/pip/wheels/1b/3a/92/28b93e2fbfdbb07509ca4d6f50c5e407f48dce4ddbda69a4ab
Successfully built pyspark


In [3]:
pip install install-jdk


In [4]:
pip install findspark

In [5]:
import random
import findspark
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from datetime import datetime, timedelta

In [8]:
# Инициализируем Spark
findspark.init()
spark = SparkSession.builder.master("local").appName("SyntheticDataGeneration").getOrCreate()

In [9]:

# Функция для генерации случайных данных
def generate_data(num_rows=1000):
    products = ['Laptop', 'Smartphone', 'Tablet', 'Smartwatch', 'Headphones']
    min_quantity = 1
    max_quantity = 5
    min_price = 50
    max_price = 2000
    current_date = datetime.now()

    data = []
    for _ in range(num_rows):
        order_date = current_date - timedelta(days=random.randint(0, 365))  # Случайная дата за последний год
        user_id = random.randint(1, 1000)  # Случайный идентификатор пользователя
        product = random.choice(products)  # Случайный продукт
        quantity = random.randint(min_quantity, max_quantity)  # Случайное количество
        price = round(random.uniform(min_price, max_price), 2)  # Случайная цена

        data.append((order_date, user_id, product, quantity, price))

    return data

# Генерация 1000 строк
data = generate_data(1000)

In [10]:
# Определяем схему для DataFrame
schema = StructType([
    StructField("OrderDate", TimestampType(), True),
    StructField("UserID", IntegerType(), True),
    StructField("Product", StringType(), True),
    StructField("Quantity", IntegerType(), True),
    StructField("Price", FloatType(), True)
])

# Создаем DataFrame из данных
df = spark.createDataFrame(data, schema=schema)
df.show(10)  # Вывод первых 10 строк для проверки

# Сохранение DataFrame в формате CSV
df.write.csv("synthetic_orders.csv", header=True, mode="overwrite")


+--------------------+------+----------+--------+-------+
|           OrderDate|UserID|   Product|Quantity|  Price|
+--------------------+------+----------+--------+-------+
|2024-06-01 11:49:...|   272|    Tablet|       5|1138.38|
|2023-12-26 11:49:...|   208|Smartphone|       5| 717.49|
|2024-05-27 11:49:...|   720|    Laptop|       2|1389.94|
|2024-02-12 11:49:...|    28|    Tablet|       4|  939.0|
|2024-04-03 11:49:...|   515|    Laptop|       3| 849.54|
|2024-08-22 11:49:...|    74|    Laptop|       4| 566.97|
|2024-01-24 11:49:...|   322|    Tablet|       3|1965.11|
|2024-01-01 11:49:...|   872|    Laptop|       4| 576.43|
|2023-12-14 11:49:...|    47|    Laptop|       5|  618.2|
|2023-12-13 11:49:...|   672|    Tablet|       2|1820.56|
+--------------------+------+----------+--------+-------+
only showing top 10 rows

